<a href="https://colab.research.google.com/github/rishank012/Capstone-Project/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import os

# Step 1: Importing and Preprocessing the Data

In [4]:
    from google.colab import files
    uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [6]:
df = pd.read_csv('dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [7]:
num_cols = [
    'ID',                  # parking lot identifier
    'Occupancy',
    'Capacity',
    'QueueLength',
    'TrafficConditionNearby',
    'IsSpecialDay'
]
df[num_cols] = (
    df[num_cols]
      .apply(lambda col: pd.to_numeric(col, errors='coerce'))
      .fillna(0)
)

In [8]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(['ID', 'Timestamp']).reset_index(drop=True)

# Feature Engineering:

In [9]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']

for col in ['QueueLength', 'TrafficConditionNearby']:
    mn = df.groupby('ID')[col].transform('min')
    mx = df.groupby('ID')[col].transform('max')
    df[f'{col}_norm'] = (df[col] - mn) / (mx - mn + 1e-6)

vt_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VT_weight'] = df['VehicleType'].map(vt_map).fillna(1.0)

### **Model 1:** *Baseline Linear Model*

In [10]:
alpha = 0.5
df['delta1'] = alpha * df['occ_rate']
df['price_m1'] = df.groupby('ID')['delta1'].cumsum().add(10)

### **Model 2:** *Demand Based Price Function*

In [11]:
a, b, c, d, e = 1.0, 0.8, 0.6, 1.2, 0.5
lam = 0.8

df['raw_d'] = (
      a * df['occ_rate']
    + b * df['QueueLength_norm']
    - c * df['TrafficConditionNearby_norm']
    + d * df['IsSpecialDay']
    + e * df['VT_weight']
)

d_min = df.groupby('ID')['raw_d'].transform('min')
d_max = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - d_min) / (d_max - d_min + 1e-6)

df['price_m2'] = 10 * (1 + lam * df['d_norm'])
df['price_m2'] = df['price_m2'].clip(lower=5, upper=20)
a, b, c, d, e = 1.0, 0.8, 0.6, 1.2, 0.5
lam = 0.8

df['raw_d'] = (
      a * df['occ_rate']
    + b * df['QueueLength_norm']
    - c * df['TrafficConditionNearby_norm']
    + d * df['IsSpecialDay']
    + e * df['VT_weight']
)

d_min = df.groupby('ID')['raw_d'].transform('min')
d_max = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - d_min) / (d_max - d_min + 1e-6)

df['price_m2'] = 10 * (1 + lam * df['d_norm'])
df['price_m2'] = df['price_m2'].clip(lower=5, upper=20)

###**Model 3:** *Competitive Pricing Model*

In [12]:
def comp_logic(row):
    if row['Occupancy'] >= row['Capacity']:
        row['price_m3'] = row['price_m2'] * 0.7
        row['suggest_reroute'] = True
    else:
        uplift = 0.1 * row['price_m2'] * (1 - row['TrafficConditionNearby_norm'])
        row['price_m3'] = row['price_m2'] + uplift
        row['suggest_reroute'] = False
    return row

df = df.apply(comp_logic, axis=1)

In [13]:
df.to_csv('parking.csv', index=False)

### **Defining Pathway Schema:**

In [14]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    ID: int
    Occupancy: int
    Capacity: int
    occ_rate: float
    QueueLength_norm: float
    TrafficConditionNearby_norm: float
    IsSpecialDay: int
    VT_weight: float
    price_m1: float
    price_m2: float
    price_m3: float
    suggest_reroute: bool

In [15]:
data = pw.demo.replay_csv(
    'parking.csv',
    schema=ParkingSchema,
    input_rate=500
)

In [16]:
fmt = '%Y-%m-%d %H:%M:%S'
stream = data.with_columns(t = data.Timestamp.dt.strptime(fmt))

## **Bokeh visualization of all three models:**

In [17]:
def multi_price_plot(source):
    fig = bokeh.plotting.figure(
        height=400, width=900,
        title='Dynamic Parking Prices: Models 1, 2 & 3',
        x_axis_type='datetime'
    )
    fig.line('t','price_m1', source=source, color='blue',  legend_label='Model 1')
    fig.line('t','price_m2', source=source, color='green',legend_label='Model 2')
    fig.line('t','price_m3', source=source, color='red',   legend_label='Model 3')
    fig.legend.location = 'top_left'
    return fig

pn.extension()
viz = stream.plot(multi_price_plot, sorting_col='t')
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [18]:
os.environ['PW_RUN_SANDBOX'] = 'true'
pw.run()

Output()